In [1]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import pickle

Essential to run all this section: load the pickl of our experiment -- make sure to change the path ;) 

You can print level on and level two keys in case you have a doubt (cell 5) one can use these list to loop over the dataset  
- Level 1 will let you chose which method you want to see: raw, or method 1, 2, or 3  (corresponding to each level of challenge)
- Level 2 which of the 6 sequences


In [2]:
def load_dict_of_dicts_pickle(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [3]:
data = load_dict_of_dicts_pickle(r"C:\Users\WillQuantique\OneDrive - univ-lyon2.fr\Fac\M1\DataChallege\git\M1_S8_dataChallenge_24\code\test_jupyter_notebook\data_for_metrics.pkl")

In [4]:
level_one_keys = list(data.keys())
level_two_keys =[]
for k in level_one_keys:
    level_two_keys.extend(list(data[k].keys()))
level_two_keys = set(level_two_keys)

In [5]:
print(level_one_keys)
print(level_two_keys)

['method_1', 'method_2', 'method_3', 'raw']
{'car_scene_2', 'scene_1', 'car_scene_3', 'scene_3', 'car_scene_1', 'scene_2'}


In [9]:

def compartive_metrics_loop_(data, method: str,function, idx:int = 0 ):
    """
    takes the name of a method (string) and a function
    will measure and return a comparative mettric betwenn raw image and filtered image of index idx
    in tha sequence.
    theresults are returned in a list
    if idx is none will compare the two complete sequences
    """

    raw_images = data["raw"]
    method = data[method]

    featured = {}
    if idx is not None:
        for k in level_two_keys:
            raw_pic = raw_images[k][idx]
            cal_pic = method[k][idx]
            featured.update({k:function(raw_pic, cal_pic)})
    else:
        for k in level_two_keys:
            raw_seq = raw_images[k]
            cal_seq = method[k]
            featured.update({k:function(raw_seq, cal_seq)})
    return featured

In [10]:
def single_metrics_loop_(data, method: str,function, idx:int = 0 ):
    """
    same as the previous one but for non comparative metrics
    """
    method = data[method]

    featured = {}
    if idx is not None:
        for k in level_two_keys:
            pic = method[k][idx]
            featured.update({k:function(pic)})
    else:
        for k in level_two_keys:
            seq = method[k]
            featured.update({k:function(seq)})
    return featured

### Metrics

Store all your metrics in this section

#### Simple metric

In [ ]:
def calculate_metrics(original_image, corrected_image):
    var = np.var(corrected_image)
    mean_pixel_value = np.mean(corrected_image)
    std_deviation = np.std(corrected_image)
    nues = (std_deviation / mean_pixel_value) * 100
    roughness = np.sum(np.abs(np.diff(corrected_image, axis=0))) + np.sum(np.abs(np.diff(corrected_image, axis=1)))
    mse = np.mean((original_image - corrected_image) ** 2)
    max_pixel_value = 2**14 - 1  # 14-bit images
    psnr = 10 * np.log10(max_pixel_value**2 / mse)

    return {
        'Variance': var,
        'NUES': nues,
        'Roughness Index': roughness,
        'PSNR': psnr
    }

#### Count feature

In [12]:
def count_features(image: np.ndarray) -> int:
    if image is None or image.size == 0:
        return 0
    if len(image.shape) != 2:
        return 0
    
    # Convert to uint8 if not already
    if image.dtype != np.uint8:
        image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    
    orb = cv2.ORB_create()
    keypoints = orb.detect(image, None)
    return len(keypoints)

def feature_ratio(raw,filtered):
    return count_features(filtered)/count_features(raw)

In [ ]:
from skimage.feature import hog
from skimage import exposure

def count_features_hog(image: np.ndarray) -> int:
    if image is None or image.size == 0:
        return 0
    if len(image.shape) != 2:
        return 0

    # Convert to uint8 if not already
    if image.dtype != np.uint8:
        image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # Compute HOG features and HOG image
    features, hog_image = hog(image, orientations=9, pixels_per_cell=(8, 8),
                              cells_per_block=(2, 2), block_norm='L2-Hys',
                              visualize=True, transform_sqrt=True, feature_vector=True)

    return len(features)

def feature_ratio_hog(raw, filtered):
    return count_features_hog(filtered) / count_features_hog(raw)

#### contrast noise ratio

In [ ]:
def contrast_to_noise_ratio(image, object_mask, background_mask):
    """
    Calculate the Contrast-to-Noise Ratio (CNR) of an image.
    
    Parameters:
    image (torch.Tensor): Input image as a PyTorch tensor.
    object_mask (torch.Tensor): Boolean mask for the object region.
    background_mask (torch.Tensor): Boolean mask for the background region.
    
    Returns:
    float: The CNR value.
    """
    # Ensure the masks are boolean tensors
    object_mask = object_mask.bool()
    background_mask = background_mask.bool()
    
    # Calculate mean intensities
    mu_object = torch.mean(image[object_mask])
    mu_background = torch.mean(image[background_mask])
    
    # Calculate standard deviation of the background
    sigma_background = torch.std(image[background_mask])
    
    # Compute CNR
    cnr = torch.abs(mu_object - mu_background) / sigma_background
    return cnr.item()


# #use
# index = 350
# scene =...
# corrected_scene = ...
# image = torch.tensor(scene[index])  # Your image tensor
# object_mask = torch.tensor(filter)  # Your object mask tensor
# background_mask = torch.tensor(corrected_scene[index])  # Your background mask tensor

# cnr_value = contrast_to_noise_ratio(image, object_mask, background_mask)
# print(f"CNR: {cnr_value}")